In [1]:
import pandas as pd
import my_utils as mu

In [2]:
df = mu.get_rawdata('GOLD')

get summonerId....


100%|██████████| 4/4 [00:02<00:00,  1.67it/s]


get puuId.....


100%|██████████| 12/12 [00:05<00:00,  2.05it/s]


get match_id....


100%|██████████| 12/12 [00:07<00:00,  1.54it/s]


get matches & timeline....


100%|██████████| 36/36 [00:53<00:00,  1.50s/it]


In [3]:
def lane_processing(tower_lane, tower_team, lane, team):
    if (tower_lane[0] == lane[0]) and (tower_team == team):
        return 1
    else:
        return 0
    

In [4]:
def assi_calc(y):
    try:
        return ','.join(list(map(lambda x: str(x), y['assistingParticipantIds'])))
    except:
        return ''

In [5]:
df_creates = []
for i in range(len(df)):
    try:
        match = df.iloc[i].matches['info']['participants']
        for j in range(len(match)):
            match_id = df.iloc[i].match_id
            game_duration = df.iloc[i].matches['info']['gameDuration']
            game_version = df.iloc[i].matches['info']['gameVersion']
            try:
                riotIdGameName = match[j]['riotIdGameName']
            except:
                riotIdGameName = ''
            try:
                riotIdTagline = match[j]['riotIdTagline']
            except:
                riotIdTagline = ''
            summoner_level = match[j]['summonerLevel']
            participants_id = match[j]['participantId']
            champion_name = match[j]['championName']
            champ_experience = match[j]['champExperience']
            team_position = match[j]['teamPosition']
            team_id = match[j]['teamId']
            win = match[j]['win']
            kills = match[j]['kills']
            deaths = match[j]['deaths']
            assists = match[j]['assists']
            total_damage_dealt = match[j]['totalDamageDealtToChampions']
            total_damage_taken = match[j]['totalDamageTaken']


            df_list = list(map(lambda x: x['events'],df.iloc[i].timelines['info']['frames']))
            event_list = [element for array in df_list for element in array]
            tower_log = list(filter(lambda x:x['type'] == 'BUILDING_KILL' ,event_list))

            try:
                ft_tower_lane = tower_log[0]['laneType']
                ft_tower_team = tower_log[0]['teamId']
            except:
                ft_tower_lane = 'n'
                ft_tower_team = 'n'
            ft = lane_processing(ft_tower_lane,ft_tower_team,team_position,team_id)

            tower_tmp = list(map(lambda x: (x['laneType'],x['teamId'],x['timestamp']) ,tower_log))

            try:
                laneTower = list(filter(lambda x:(x[0][0] == team_position[0]) & (x[1] == team_id) ,tower_tmp))[0]
                lane_flag = 1
                laneTowerTime = laneTower[-1]
            except:
                lane_flag = 0
                laneTowerTime = 0
            
            blue_ban = list(map(lambda x: str(x['championId']) ,df.iloc[i].matches['info']['teams'][0]['bans']))
            red_ban = list(map(lambda x: str(x['championId']) ,df.iloc[i].matches['info']['teams'][1]['bans']))
            ban_list = '|'.join(list(set(blue_ban + red_ban)))
            
            
            kill_log = list(map(lambda x: list(filter(lambda z: z['type']=='CHAMPION_KILL',x['events'])),df.iloc[i].timelines['info']['frames']))
            kill_log = [element for array in kill_log for element in array]
            k = '|'.join(list(map(lambda x : str(x['killerId']) , kill_log)))
            v = '|'.join(list(map(lambda x :str(x['victimId']), kill_log)))
            a = '|'.join(list(map(lambda x : assi_calc(x) ,kill_log)))
            
            try:
                g15 = df.iloc[i].timelines['info']['frames'][15]['participantFrames'][(str(j+1))]['totalGold']
            except:
                g15 = 0
        
            df_creates.append([match_id,game_duration,game_version,riotIdGameName,riotIdTagline,summoner_level,participants_id,
                              champion_name,champ_experience,team_position,team_id,win,kills,deaths,assists,total_damage_dealt,total_damage_taken,
                              ft,lane_flag,laneTowerTime,ban_list,k,v,a,g15])
    except:
        continue
columns =['matchId', 'gameDuration', 'gameVersion', 'riotIdGameName','riotIdTagline', 
           'summonerLevel', 'participantId', 'championName', 'champExperience', 
           'teamPosition', 'teamId', 'win', 'kills', 'deaths', 'assists', 'totalDamageDealtToChampions', 'totalDamageTaken',
         'firstDT','laneTower','LaneTowerTime','bans','killerId','victimId','assistId','G15']
df2 = pd.DataFrame(df_creates , columns = columns)

In [6]:
df2

,matchId,gameDuration,gameVersion,riotIdGameName,riotIdTagline,summonerLevel,participantId,championName,champExperience,teamPosition,...,totalDamageDealtToChampions,totalDamageTaken,firstDT,laneTower,LaneTowerTime,bans,killerId,victimId,assistId,G15
0,KR_6869818117,978,13.24.547.9214,김태백,KR1,547,1,Jax,8980,TOP,...,10820,9610,0,0,0,81|104|84|99|51|114|166|555|777|41,2|3|1|2|9|4|10|2|1|3|9|9|1|3|1|3|4|1|4|3|4,8|8|7|6|5|9|4|10|6|6|4|5|9|7|8|10|9|6|10|8|7,"3||2|1|10|2,5|9|4|2|2|7,10|7,10|2,4,5|1,2,4,5|...",5826
1,KR_6869818117,978,13.24.547.9214,A 7,7777,295,2,Nocturne,6909,JUNGLE,...,8707,14814,0,0,0,81|104|84|99|51|114|166|555|777|41,2|3|1|2|9|4|10|2|1|3|9|9|1|3|1|3|4|1|4|3|4,8|8|7|6|5|9|4|10|6|6|4|5|9|7|8|10|9|6|10|8|7,"3||2|1|10|2,5|9|4|2|2|7,10|7,10|2,4,5|1,2,4,5|...",6364
2,KR_6869818117,978,13.24.547.9214,지금은아니잖아,KR1,96,3,Irelia,9685,MIDDLE,...,11303,9247,0,0,0,81|104|84|99|51|114|166|555|777|41,2|3|1|2|9|4|10|2|1|3|9|9|1|3|1|3|4|1|4|3|4,8|8|7|6|5|9|4|10|6|6|4|5|9|7|8|10|9|6|10|8|7,"3||2|1|10|2,5|9|4|2|2|7,10|7,10|2,4,5|1,2,4,5|...",6530
3,KR_6869818117,978,13.24.547.9214,바지속의느낌표,KR1,298,4,Veigar,5769,BOTTOM,...,6763,5398,0,0,0,81|104|84|99|51|114|166|555|777|41,2|3|1|2|9|4|10|2|1|3|9|9|1|3|1|3|4|1|4|3|4,8|8|7|6|5|9|4|10|6|6|4|5|9|7|8|10|9|6|10|8|7,"3||2|1|10|2,5|9|4|2|2|7,10|7,10|2,4,5|1,2,4,5|...",5079
4,KR_6869818117,978,13.24.547.9214,석기시대 집도둑,KR1,168,5,Senna,5369,UTILITY,...,2429,4354,0,0,0,81|104|84|99|51|114|166|555|777|41,2|3|1|2|9|4|10|2|1|3|9|9|1|3|1|3|4|1|4|3|4,8|8|7|6|5|9|4|10|6|6|4|5|9|7|8|10|9|6|10|8|7,"3||2|1|10|2,5|9|4|2|2|7,10|7,10|2,4,5|1,2,4,5|...",3361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,KR_6837920509,1403,13.24.547.9214,모솔종이인형,KR1,197,6,Jax,13617,TOP,...,18127,14737,0,1,1069011,518|517|-1|15|56|360|89|157|238|420,9|4|4|6|9|1|6|5|6|9|4|6|1|3|10|9|8|7|9|2|10|8|...,5|9|10|1|2|6|1|10|1|4|9|2|6|7|5|4|5|2|4|6|2|4|...,"10||||7,8||7|3,4||7,10|5|7|2,3|1|9|10|10|8,9,1...",5941
356,KR_6837920509,1403,13.24.547.9214,에버소울,1111,287,7,Amumu,10737,JUNGLE,...,11202,20444,0,0,0,518|517|-1|15|56|360|89|157|238|420,9|4|4|6|9|1|6|5|6|9|4|6|1|3|10|9|8|7|9|2|10|8|...,5|9|10|1|2|6|1|10|1|4|9|2|6|7|5|4|5|2|4|6|2|4|...,"10||||7,8||7|3,4||7,10|5|7|2,3|1|9|10|10|8,9,1...",5848
357,KR_6837920509,1403,13.24.547.9214,windy0730,5432,140,8,Veigar,12408,MIDDLE,...,19024,7763,0,0,0,518|517|-1|15|56|360|89|157|238|420,9|4|4|6|9|1|6|5|6|9|4|6|1|3|10|9|8|7|9|2|10|8|...,5|9|10|1|2|6|1|10|1|4|9|2|6|7|5|4|5|2|4|6|2|4|...,"10||||7,8||7|3,4||7,10|5|7|2,3|1|9|10|10|8,9,1...",6344
358,KR_6837920509,1403,13.24.547.9214,최형철의검은생각,KR1,459,9,Vayne,10128,BOTTOM,...,14181,10339,0,1,958307,518|517|-1|15|56|360|89|157|238|420,9|4|4|6|9|1|6|5|6|9|4|6|1|3|10|9|8|7|9|2|10|8|...,5|9|10|1|2|6|1|10|1|4|9|2|6|7|5|4|5|2|4|6|2|4|...,"10||||7,8||7|3,4||7,10|5|7|2,3|1|9|10|10|8,9,1...",6653


In [7]:
blue_team = df2[df2.teamId == 100]
red_team = df2[df2.teamId == 200]

In [8]:
tmp = red_team[['matchId','gameVersion','teamPosition','championName','G15']].rename(columns = {'championName':'enemy_champ', 'G15':'enemy_G15'})

In [9]:
blue_team_merge = pd.merge(blue_team,tmp,on = ['matchId','gameVersion','teamPosition'])

In [10]:
tmp_blue = blue_team[['matchId','gameVersion','teamPosition','championName','G15']].rename(columns = {'championName':'enemy_champ', 'G15':'enemy_G15'})

In [11]:
red_team_merge = pd.merge(red_team,tmp_blue,on = ['matchId','gameVersion','teamPosition'])

In [12]:
team_df = pd.concat([blue_team_merge,red_team_merge])

In [15]:
team_df['win'] = team_df.apply(lambda x :1 if x.win == True else 0,axis=1)

In [17]:
tmp_df = team_df[['matchId','championName','enemy_champ','teamPosition','win']]

In [27]:
tmp_df

,matchId,championName,enemy_champ,teamPosition,win
0,KR_6869818117,Jax,TahmKench,TOP,1
1,KR_6869818117,Nocturne,LeeSin,JUNGLE,1
2,KR_6869818117,Irelia,Sylas,MIDDLE,1
3,KR_6869818117,Veigar,Samira,BOTTOM,1
4,KR_6869818117,Senna,Yuumi,UTILITY,1
...,...,...,...,...,...
175,KR_6837920509,Jax,Trundle,TOP,1
176,KR_6837920509,Amumu,Graves,JUNGLE,1
177,KR_6837920509,Veigar,Azir,MIDDLE,1
178,KR_6837920509,Vayne,Nilah,BOTTOM,1


In [29]:
tmp_df[['championName','enemy_champ','teamPosition','win']].groupby(['championName','teamPosition']).count()

enemy_champ  win
championName teamPosition                  
Aatrox       TOP                     3    3
             UTILITY                 1    1
Ahri         MIDDLE                  3    3
Akali        MIDDLE                  1    1
             TOP                     2    2
...                                ...  ...
Zeri         BOTTOM                  1    1
Ziggs        BOTTOM                  1    1
             MIDDLE                  1    1
Zoe          MIDDLE                  2    2
Zyra         UTILITY                 3    3

[149 rows x 2 columns]

In [40]:
cnt = tmp_df[['championName','enemy_champ','teamPosition','win']].groupby(['championName','enemy_champ','teamPosition']).count()

In [41]:
cnt = cnt.rename(columns = {'win':'cnt'})

In [42]:
cnt

cnt
championName enemy_champ teamPosition     
Aatrox       Fiora       TOP             1
             Garen       TOP             1
             Sett        TOP             1
             Thresh      UTILITY         1
Ahri         Akali       MIDDLE          1
...                                    ...
Zoe          Viktor      MIDDLE          1
             Zed         MIDDLE          1
Zyra         Lux         UTILITY         1
             Soraka      UTILITY         1
             Xerath      UTILITY         1

[344 rows x 1 columns]

In [23]:
tmp_df[['championName','teamPosition','win']].groupby(['championName','teamPosition']).sum()

win
championName teamPosition     
Aatrox       TOP             2
             UTILITY         1
Ahri         MIDDLE          3
Akali        MIDDLE          0
             TOP             2
...                        ...
Zeri         BOTTOM          0
Ziggs        BOTTOM          0
             MIDDLE          0
Zoe          MIDDLE          1
Zyra         UTILITY         3

[149 rows x 1 columns]

In [32]:
cnt = tmp_df[['championName','teamPosition','win']].groupby(['championName','teamPosition']).count()

In [34]:
cnt = cnt.rename({'win':'cnt'})

In [25]:
cnt_win = tmp_df[['championName','enemy_champ','teamPosition','win']].groupby(['championName','enemy_champ','teamPosition']).sum()

In [26]:
cnt_win

win
championName enemy_champ teamPosition     
Aatrox       Fiora       TOP             1
             Garen       TOP             0
             Sett        TOP             1
             Thresh      UTILITY         1
Ahri         Akali       MIDDLE          1
...                                    ...
Zoe          Viktor      MIDDLE          1
             Zed         MIDDLE          0
Zyra         Lux         UTILITY         1
             Soraka      UTILITY         1
             Xerath      UTILITY         1

[344 rows x 1 columns]

In [44]:
result = cnt.join(cnt_win)

In [48]:
result.sort_values(by = 'win',ascending = False)

cnt  win
championName enemy_champ teamPosition          
Yone         Sylas       MIDDLE          2    2
LeeSin       Poppy       JUNGLE          2    2
Caitlyn      Tristana    BOTTOM          2    2
Rakan        Thresh      UTILITY         2    2
KogMaw       MissFortune BOTTOM          2    2
...                                    ...  ...
Nocturne     Maokai      JUNGLE          1    0
             Warwick     JUNGLE          1    0
Fiora        Aatrox      TOP             1    0
FiddleSticks Nunu        JUNGLE          1    0
Ezreal       Sivir       BOTTOM          1    0

[344 rows x 2 columns]

In [ ]:
def get_match_timeline_df(df):
    # df를 한개로 만들기
    df_creater = []
    print('소환사 스텟 생성중.....')
    for i in tqdm(range(len(df))):       
        # matches 관련된 데이터 
        try:
            if df.iloc[i].matches['info']['gameDuration'] > 900:   # 게임시간이 15분이 안넘을경우에는 패스하기
                for j in range(10):
                    tmp = []
                    tmp.append(df.iloc[i].match_id)
                    tmp.append(df.iloc[i].matches['info']['gameDuration'])
                    tmp.append(df.iloc[i].matches['info']['gameVersion'])
                    tmp.append(df.iloc[i].matches['info']['participants'][j]['summonerName'])
                    tmp.append(df.iloc[i].matches['info']['participants'][j]['summonerLevel'])
                    tmp.append(df.iloc[i].matches['info']['participants'][j]['participantId'])
                    tmp.append(df.iloc[i].matches['info']['participants'][j]['championName'])
                    tmp.append(df.iloc[i].matches['info']['participants'][j]['champExperience'])
                    tmp.append(df.iloc[i].matches['info']['participants'][j]['teamPosition'])
                    tmp.append(df.iloc[i].matches['info']['participants'][j]['teamId'])
                    tmp.append(df.iloc[i].matches['info']['participants'][j]['win'])
                    tmp.append(df.iloc[i].matches['info']['participants'][j]['kills'])
                    tmp.append(df.iloc[i].matches['info']['participants'][j]['deaths'])
                    tmp.append(df.iloc[i].matches['info']['participants'][j]['assists'])
                    tmp.append(df.iloc[i].matches['info']['participants'][j]['totalDamageDealtToChampions'])
                    tmp.append(df.iloc[i].matches['info']['participants'][j]['totalDamageTaken'])
            #timeline 관련된 데이터
                    for k in range(5,26):
                        try:
                            tmp.append(df.iloc[i].timeline['info']['frames'][k]['participantFrames'][str(j+1)]['totalGold'])
                        except:
                            tmp.append(0)
                    df_creater.append(tmp)
        except:
            print(i)
            continue
    columns = ['gameId','gameDuration','gameVersion','summonerName','summonerLevel','participantId','championName','champExperience',
    'teamPosition','teamId','win','kills','deaths','assists','totalDamageDealtToChampions','totalDamageTaken','g_5','g_6','g_7','g_8','g_9','g_10','g_11','g_12','g_13','g_14','g_15','g_16','g_17',
    'g_18','g_19','g_20','g_21','g_22','g_23','g_24','g_25']
    df = pd.DataFrame(df_creater,columns = columns).drop_duplicates()
    print('df 제작이 완료되었습니다. 현재 df의 수는 %d 입니다'%len(df))
    return df

In [52]:
query_mysql = """
create table lol_datas(gameId varchar(30), gameDuration int, gameVersion varchar(40), summonerName varchar(50),
summonerLevel int, participantId int, championName varchar(50), champExperience int,
teamPosition varchar(50), teamId int, win varchar(50), kills int, deaths int, assists int,
totalDamageDealtToChampions int ,totalDamageTaken int, g_5 int,g_6 int,g_7 int,
g_8 int,g_9 int,g_10 int,g_11 int,g_12 int,g_13 int,g_14 int,
g_15 int,g_16 int,g_17 int,g_18 int,g_19 int,g_20 int,g_21 int,
g_22 int,g_23 int, g_24 int,g_25 int
)
"""

In [57]:
conn = mu.connect_mysql()

In [54]:
mu.sql_execute(conn,query_mysql)

()

In [55]:
conn.commit()

In [56]:
conn.close()

In [59]:
query_mysql2 = 'ALTER TABLE lol_datas add primary key (gameId,participantId)'

In [60]:
mu.sql_execute(conn,query_mysql2)

()

In [61]:
conn.commit()

In [62]:
def insert_matches_timeline_mysql(row,conn):
    # lambda를 이용해서 progress_apply를 통해 insert할 구문 만들기
    query = (
             f'insert into lol_datas(gameId, gameDuration, gameVersion, summonerName, summonerLevel, participantId,'
             f'championName, champExperience, teamPosition, teamId, win, kills, deaths, assists,'
             f'totalDamageDealtToChampions, totalDamageTaken, g_5, g_6, g_7, g_8, g_9, g_10, g_11, g_12 ,g_13,g_14,'
             f'g_15, g_16, g_17, g_18, g_19, g_20, g_21, g_22, g_23, g_24, g_25)'
             f'values(\'{row.gameId}\',{row.gameDuration}, \'{row.gameVersion}\', \'{row.summonerName}\','
             f'{row.summonerLevel}, {row.participantId},\'{row.championName}\',{row.champExperience},'
             f'\'{row.teamPosition}\', {row.teamId}, \'{row.win}\', {row. kills}, {row.deaths}, {row.assists},'
             f'{row.totalDamageDealtToChampions},{row.totalDamageTaken},{row.g_5},{row.g_6},{row.g_7},{row.g_8},'
             f'{row.g_9},{row.g_10},{row.g_11},{row.g_12},{row.g_13},{row.g_14},{row.g_15},{row.g_16},{row.g_17},'
             f'{row.g_18},{row.g_19},{row.g_20},{row.g_21},{row.g_22},{row.g_23},{row.g_24},{row.g_25})'
             f'ON DUPLICATE KEY UPDATE '
             f'gameId = \'{row.gameId}\', gameDuration = {row.gameDuration}, gameVersion = \'{row.gameVersion}\', summonerName= \'{row.summonerName}\','
             f'summonerLevel = {row.summonerLevel},participantId = {row.participantId},championName = \'{row.championName}\','
             f'champExperience = {row.champExperience}, teamPosition = \'{row.teamPosition}\', teamId = {row.teamId},win = \'{row.win}\','
             f'kills = {row. kills}, deaths = {row.deaths}, assists = {row.assists}, totalDamageDealtToChampions = {row.totalDamageDealtToChampions},'
             f'totalDamageTaken = {row.totalDamageTaken},g_5 = {row.g_5},g_6 = {row.g_6},g_7 = {row.g_7},g_8 = {row.g_8},g_9 = {row.g_9},'
             f'g_10 = {row.g_10},g_11 = {row.g_11},g_12 = {row.g_12},g_13 = {row.g_13},g_14 = {row.g_14},g_15 = {row.g_15},g_16 = {row.g_16},g_17 = {row.g_17},'
             f'g_18 = {row.g_18},g_19 = {row.g_19},g_20 = {row.g_20},g_21 = {row.g_21},g_22 = {row.g_22},g_23 = {row.g_23},g_24 = {row.g_24},g_25 = {row.g_25}'
            )
    mysql_execute(query,conn)
    return query


In [73]:
import my_utils as mu
import pandas as pd

In [87]:
conn = mu.connect_mysql()
query = 'select * from lol_datas'
result = mu.sql_execute_dict(conn,query)
conn.close()

In [88]:
result_df = pd.DataFrame(result)

In [89]:
result_df = result_df[(result_df.gameDuration > 900)]

In [90]:
sample = result_df[['gameId','championName','win','teamPosition','g_15','teamId']]

In [91]:
sample

,gameId,championName,win,teamPosition,g_15,teamId
0,KR_6666594486,Sett,False,TOP,5416,100
1,KR_6666594486,Taliyah,False,JUNGLE,6279,100
2,KR_6666594486,Zed,False,MIDDLE,6127,100
3,KR_6666594486,Sivir,False,BOTTOM,6752,100
4,KR_6666594486,Nautilus,False,UTILITY,3833,100
...,...,...,...,...,...,...
2599,KR_6870041041,Aatrox,True,TOP,4504,200
2600,KR_6870041041,Talon,True,JUNGLE,6305,200
2601,KR_6870041041,Zac,True,MIDDLE,4671,200
2602,KR_6870041041,Lucian,True,BOTTOM,4979,200


- 챔피언 1vs1 라인전 승률 : G15의 데이터를 바탕으로 승률을 구하기 
- 챔피언 1vs1 게임 승률 : G15가 아니라 WIN의 데이터를 바탕으로 승률을 구하기

In [216]:
result_df.sort_values('games',ascending = False)

lane_win_cnt  win_cnt  games
championName teamPosition enemy_champ                              
Kaisa        BOTTOM       Ezreal                 19       23     41
Ezreal       BOTTOM       Kaisa                  22       18     41
LeeSin       JUNGLE       JarvanIV               15        8     23
JarvanIV     JUNGLE       LeeSin                  8       15     23
Jinx         BOTTOM       Ezreal                 10       13     23
...                                             ...      ...    ...
Kled         TOP          Viktor                  0        1      1
                          Vladimir                0        1      1
             UTILITY      Diana                   1        1      1
KogMaw       BOTTOM       Jhin                    0        0      1
Zyra         UTILITY      Yuumi                   1        1      1

[4792 rows x 3 columns]

In [ ]:
result_df.sort_values('games',ascending = False)